## 引言
前文[微调方法概览](https://golfxiao.blog.csdn.net/article/details/140859269)总结了微调的各种方法，并且在更前面两篇文章[欺诈文本分类微调（六）：Lora单卡](https://golfxiao.blog.csdn.net/article/details/141440847)
和[欺诈文本分类微调（七）—— lora单卡二次调优](https://golfxiao.blog.csdn.net/article/details/141500352)中已经尝试过用Lora进行微调，本文出于好奇准备尝试下用QLora进行微调的效果。

QLoRA是一种新的微调大型语言模型（LLM）的方法，它的特点是能在节省内存的同时保持推理性能。它的出现是为了应对大型模型微调时内存需求大，成本昂贵的问题。

工作原理：首先将LLM进行4位量化，从而显著减少模型的内存占用，接着使用低阶适配器（LoRA）方法对量化的LLM进行微调，因此，QLora可以看成是量化+Lora的结合体。

具有以下核心技术：
- 4位量化，它创新性的引入了特殊的4位浮点数表示方法NF4(Normal Float 4-bit)，使用非均匀量化来平衡数据范围与精度。
- 双量化，一种对量化后常数再次进行量化的方法，每个参数可平均节省约 0.37 位。
- 分页优化，使用具有 NVIDIA 统一内存的分页优化器，以避免具有长序列长度的小批量时出现内存峰值。1



依赖库安装：
```
pip install -q -U bitsandbytes

> bitsandbytes主要是针对llm和transformers模型提供了优化和量化模型的功能，专门为8位优化器、矩阵乘法和量化而设计，提供了像8位Adam/AdamW之类的函数。目标是通过8位操作实现高效的计算和内存使用从而使llm更易于访问。```

本文的用欺诈文本分类这个业务场景来目的是实际验证下QLora进行量化微调。效果，

## QLora训练

#### 初始化
引入之前封装好的trainer.py脚本，定义模型路径和数据集路径，以及要使用的GPU设备。

In [1]:
%run trainer.py

In [2]:
traindata_path = '/data2/anti_fraud/dataset/train0819.jsonl'
evaldata_path = '/data2/anti_fraud/dataset/eval0819.jsonl'
model_path = '/data2/anti_fraud/models/modelscope/hub/Qwen/Qwen2-1___5B-Instruct'
output_path = '/data2/anti_fraud/models/Qwen2-1___5B-Instruct_ft_0830_2'

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = 'cuda'

#### 加载模型和数据集

由于QLora需要以量化的方式来加载模型，所以加载模型的方法需要作调整，这里的改动是引入`BitsAndBytesConfig`类构建一个量化配置`quantization_config`, 具体配置项释义：
- load_in_4bit：决定了模型参数以4位量化格式加载，加载后的模型参数占用空间会比较小； 
- bnb_4bit_compute_dtype=bfloat16：决定了矩阵乘法的计算精度使用bfloat16，输入数据也会被转换成bfloat16位进行计算； 
- bnb_4bit_quant_type：指定量化数据类型nf4； 
- bnb_4bit_use_double_quant：是否启用双重量化； 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

def load_model(model_path, device='cuda'):
    tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=False, 
            bnb_4bit_quant_type='nf4'
        ),
    )
    return model, tokenizer

> 注1：普通的量化通常是将数值分成均匀的区间，比如，将0到1之间的数值分成16个区间，每个区间的宽度相同。而NF4则根据数据的分布情况，使用不均匀的区间来表示数值，这样可以更有效地表示模型中的重要数值，特别是那些频繁出现的数值。

> 注2：双重量化是指在已经量化的基础上再进行量化，第二次量化并不会改变位数本身（即仍然是4位），它的目的是通过更紧凑地表示数值，使得存储和计算更加高效。由于每一次量化都会引入一些量化误差，双重量化可能会带来更大的数值误差，所以一般只用于极端内存受限的情况下。

> 注3：之所以模型参数加载使用4位而计算时使用16位，是因为量化本身已经带来了误差，计算时需要采用更高的精度是为了减少量化误差带来的影响。

In [ ]:
加载模型参数和token序列化器。

In [5]:
%%time
model, tokenizer = load_model(model_path, device)
model.device

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


CPU times: user 1.14 s, sys: 574 ms, total: 1.72 s
Wall time: 1.81 s


device(type='cuda', index=0)

In [11]:
> 通过统计的耗时信息可以看到，使用量化方式加载模型的过程是比较慢的，耗时了1分52秒，中间涉及到将模型参数从高精度的浮点数（如 FP32）转换为低精度的 NF4 格式。与之对比，不带量化时基本是秒加载。

加载数据集，复用前文的方法。

SyntaxError: invalid character '，' (U+FF0C) (3210621457.py, line 1)

In [7]:
train_dataset, eval_dataset = load_dataset(traindata_path, evaldata_path, tokenizer)

#### 构建训练参数

训练参数：引入分页内存优化器来优化训练过程中的内存分配。

In [8]:
train_args = build_train_arguments(output_path)
train_args.optim="paged_adamw_32bit"   

> paged_adamw_32bit 是一种优化器配置，它使用了分页内存管理和32位浮点数来优化训练过程，可以帮助你在训练大规模模型时更有效地管理内存和计算资源。


lora配置：同[前文Lora训练](https://golfxiao.blog.csdn.net/article/details/141500352)一样使用大小为16的秩。

In [9]:
lora_config = build_loraconfig()
lora_config.lora_dropout = 0.2   
lora_config.r = 16
lora_config.lora_alpha = 32

#### 开始训练

构造训练器开始训练。

In [10]:
trainer = build_trainer(model, tokenizer, train_args, lora_config, train_dataset, eval_dataset)
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 18,464,768 || all params: 1,562,179,072 || trainable%: 1.1820


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
100,0.046700,0.029243
200,0.041400,0.041580
300,0.016300,0.025357
400,0.027700,0.022728
500,0.023300,0.022268
600,0.028900,0.021204
700,0.020000,0.019347
800,0.019000,0.022994
900,0.018700,0.019503
1000,0.016300,0.018722


TrainOutput(global_step=1900, training_loss=0.035359279932944396, metrics={'train_runtime': 2384.4155, 'train_samples_per_second': 23.637, 'train_steps_per_second': 1.477, 'total_flos': 8.369815519690752e+16, 'train_loss': 0.035359279932944396, 'epoch': 1.61839863713799})

In [ ]:
训练过程中观察内存变化：22G+。

训练时的显存占用相比非量化时并没有明显变化，基本上占满了24G显卡的显存。

推测原因可能是：QLoRA只是通过量化技术减少了模型参数加载时的显存占用，但训练时仍然会反量化为16位进行矩阵计算，尤其是前向和反向传播阶段，显存的主要消耗来自于激活值、梯度和优化器状态，模型参数仅仅是一小部分，这就导致真正训练过程中占用的显存相比非量化时并没有减少。

QLoRA 主要通过量化模型参数来减小显存占用，但在需要更大的 batch size 的场景下，其显存优化效果可能并不显著。

## 评估测试
用验证损失最低的checkpoint-2200进行测评。

In [12]:
%run evaluate.py
checkpoint_path='/data2/anti_fraud/models/Qwen2-1___5B-Instruct_ft_0830_2/checkpoint-1700'
evaluate(model_path, checkpoint_path, evaldata_path, device, batch=True, debug=True)

progress: 100%|██████████| 2348/2348 [02:31<00:00, 15.54it/s]

tn：1143, fp:22, fn:215, tp:968
precision: 0.9777777777777777, recall: 0.8182586644125106, accuracy: 0.8990630323679727


这个训练结果和前文的召回率`0.86`区别不大，说明使用量化后的模型参数进行训练确实能保持和16位精度的参数训练几乎一样的效果。

**小结**：本文通过实际训练来测试QLora对于显存占用和推理性能方面的效果，在我们这个验证结果里，推理性能方面几乎可以保持同先前一样的效果，但显存占用只在加载时降低了不到1/2, 而在训练过程中相比于非量化时没有明显减少。原因可能是由于我们的模型太小，训练时的批量相对较大，所以模型参数加载时所优化的那部分内存，与整个训练过程中所占用的内存相比比例较低，内存优化的整体效果就不明显。基于QLora主要是通过减少模型参数所占用的显存这个原理出发，个人理解可能在参数更大的模型和batch_size更小的训练时效果可能会有所提升。

## 相关阅读
- [微调方法概览](https://golfxiao.blog.csdn.net/article/details/140859269)
- [欺诈文本分类微调（六)：Lora单卡训练](https://golfxiao.blog.csdn.net/article/details/141440847)
- [欺诈文本分类微调（七)：lora单卡二次调优](https://golfxiao.blog.csdn.net/article/details/141500352)
- [QLoRA量化微调策略与实践](https://blog.csdn.net/FrenzyTechAI/article/details/132686051)